In [1]:
import pandas as pd
import os 
import numpy as np
import datetime
import folium 
import plotly.express as px
import plotly
import plotly.graph_objects as go
import argparse
import pyarrow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from functools import reduce
from datetime import datetime, timedelta

In [2]:
folder_path = "/Users/ruiwenzhang/Dropbox/PSV Rider Feedback/Data/Sensor Data/FinalData"
outputs_path = "/Users/ruiwenzhang/Desktop/Kenya/Weekly_report_test"
sacco_path = "/Users/ruiwenzhang/Dropbox/PSV Rider Feedback/Data/Matatu Sensor Installation Survey/FinalData"
if not os.path.exists(outputs_path):
                os.mkdir(outputs_path)
sensor_tracing_path = os.path.join(folder_path, "sensortracing.gz.parquet")
eco_data_path = os.path.join(folder_path, "echodriving.gz.parquet")
survey_path = os.path.join(sacco_path, "psv_sensor_installation.csv")

In [ ]:
class weekly_report:
    def __init__(self,
                data_path,
                sacco_path,
                outputs_path,
                year,
                month,
                date
                ):
        
        self.data_path = data_path
        self.eco_data_path = os.path.join(self.data_path, "echodriving.gz.parquet")
        self.sensor_tracing_path = os.path.join(self.data_path, "sensortracing.gz.parquet")
        self.sacco_path = sacco_path
        self.survey_path = os.path.join(self.sacco_path, "psv_sensor_installation.csv")
        self.outputs_path = outputs_path
        if not os.path.exists(self.outputs_path):
                os.mkdir(self.outputs_path)
        
        self.year = year
        self.month = month
        self.date = date
    
        print('Loading data...')
        self.load_data()
        self.past_twowk_agg()
    
    
    def load_data(self):
        
        def data_process_sensor(df):
            #df = df[df.speed_kmhr > 0]
            df.time_str = pd.to_datetime(df.time_str,format = "%Y-%m-%d %H:%M:%S")
            df["date"] = df.time_str.dt.date
            df["hr"] = df.time_str.dt.hour
            #df['time_delta'] = df.groupby(['reg_no'])['time_str'].diff()
            #df['time_delta'] = df['time_delta'].apply(lambda x: x.total_seconds())
            df['day_hr']= df['time_str'].dt.floor('h')
            df["reg_no"] = df["reg_no"].str.split(" ", expand=True)[1] + " " + df["reg_no"].str.split(" ", expand=True)[2]
            df = df.sort_values('speed_kmhr').\
                 drop_duplicates(["reg_no","time_str","latitude","longitude"], keep='last')
            
            # find the first date and last date that each vehicle is online
            first_online_date = df.sort_values(["reg_no","date"]).groupby('reg_no').\
            head(1)[["reg_no","date"]].rename(columns = {'date': '1st_oldate'})
            last_online_date = df.sort_values(["reg_no","date"]).groupby('reg_no').\
            tail(1)[["reg_no","date"]].rename(columns = {'date': 'last_oldate'})
            online_date_sen = first_online_date.merge(last_online_date)
            
            df = df.sort_values('date')
            first_date = df.date.unique()[0]
            last_date = df.date.unique()[-1]
            num_days = abs(last_date - first_date).days + 1
            num_weeks = (num_days // 7) + 1 if (num_days % 7) != 0 else num_days // 7
            week_bin = pd.date_range(df['date'].unique()[0], periods = num_weeks + 1, freq = '168H')
            labels = [i for i in range(1,len(week_bin+1))]
            df["week"] = pd.cut(df['date'].astype('datetime64[ns]'), 
                          week_bin, include_lowest = True, labels = labels)
            
            df = df.merge(online_date_sen, how = "outer", on = 'reg_no')
            df["n_weeks"] = df.groupby("reg_no")['week'].transform(lambda x: x.nunique())
            
            nozero_spd_df = df[df.speed_kmhr > 0]
    
    
            return nozero_spd_df, df
    
        
        def data_process_eco(df):
            # df = df.drop(['violation'], axis=1)
            df["violation"] = df["violation"].map(lambda x: x.lower())
            #df.Beginning = pd.to_datetime(df.Beginning.str.replace(".","-"), format = "%d-%m-%Y %H:%M:%S")
            df["begin_date"] = df.begin_datetime.dt.date
            df["begin_hour"] = df.begin_datetime.dt.hour
            #df.End = pd.to_datetime(df.End.str.replace(".","-"), format = "%d-%m-%Y %H:%M:%S")
            df["end_date"] = df.end_datetime.dt.date
            df["end_hour"] = df.end_datetime.dt.hour
            df['day_hr']= df['begin_datetime'].dt.floor('h')
            df["harsh_event"] = df.violation.str.split(":", expand=True)[0]
            df["reg_no"] = df["reg_no"].str.split(" ", expand=True)[1] + " " + df["reg_no"].str.split(" ", expand=True)[2]
            #df["level"] = df.violation.str.split(" ", expand=True)[1]
            
            df = df.sort_values('value_g').\
                 drop_duplicates(["reg_no","begin_datetime","end_datetime",
                                           "latitude_begin","longitude_begin",
                                           "latitude_end","longitude_end"], keep='last')
            df = df[df.value_g > 0.3]
            df["level"] = ["extreme" if x > 0.4 else "medium" for x in df['value_g']]
            df["violation"] = df["harsh_event"] + ": " + df["level"]
           
            #find the first date and last date that each vehicle is online
            first_online_date = df.sort_values(["reg_no","begin_date"]).groupby('reg_no').\
            head(1)[["reg_no","begin_date"]].rename(columns = {'begin_date': '1st_oldate'})
            last_online_date = df.sort_values(["reg_no","begin_date"]).groupby('reg_no').\
            tail(1)[["reg_no","begin_date"]].rename(columns = {'begin_date': 'last_oldate'})
            online_date_eco = first_online_date.merge(last_online_date)
            
            df = df.sort_values('begin_date')
            first_date = df.begin_date.unique()[0]
            last_date = df.begin_date.unique()[-1]
            num_days = abs(last_date - first_date).days + 1
            num_weeks = (num_days // 7) + 1 if (num_days % 7) != 0 else num_days // 7
            week_bin = pd.date_range(df['begin_date'].unique()[0], periods = num_weeks + 1, freq = '168H')
            labels = [i for i in range(1,len(week_bin+1))]
            df["week"] = pd.cut(df['begin_date'].astype('datetime64[ns]'), 
                          week_bin, include_lowest = True, labels = labels)
            
            df = df.merge(online_date_eco, how = "outer", on = 'reg_no')
            df["n_weeks"] = df.groupby("reg_no")['week'].transform(lambda x: x.nunique())
   
    
    
            return df
        
    
    
    
        self.sensor_tracing_raw = pd.read_parquet(self.sensor_tracing_path)
        self.sensor_data, self.sensor_data2 = data_process_sensor(self.sensor_tracing_raw)
        
        self.eco_data_raw = pd.read_parquet(self.eco_data_path)
        self.eco_data = data_process_eco(self.eco_data_raw)
        
        self.survey_data = pd.read_csv(self.survey_path)
        self.survey_data = self.survey_data[["matatu_regno", "matatu_sacco"]].drop_duplicates()
        self.survey_data["matatu_regno"] = self.survey_data["matatu_regno"].str.lower()
        
        self.sensor_data = self.sensor_data.merge(self.survey_data, how = "left", left_on = "reg_no", right_on = "matatu_regno")
        self.eco_data = self.eco_data.merge(self.survey_data, how = "left", left_on = "reg_no", right_on = "matatu_regno")
        
        self.totalvio_wk = self.eco_data.groupby(["reg_no","week"]).size().to_frame('count').reset_index()
        
        self.extvio_df = self.eco_data[self.eco_data.level == "extreme"]
        self.extvio_wk = self.extvio_df.groupby(["reg_no","week"]).size().to_frame('count').reset_index()
        
        self.sensor_data2 = self.sensor_data.sort_values(["reg_no","time_str"])
        self.sensor_data2 = self.sensor_data2.loc[self.sensor_data2.groupby(["reg_no","date"]).speed_kmhr.cumsum() != 0]
        self.sensor_data2 = self.sensor_data2.sort_values(["reg_no","time_str"], ascending = False)
        self.sensor_data2 = self.sensor_data2.loc[self.sensor_data2.groupby(["reg_no","date"]).speed_kmhr.cumsum() != 0]
        self.sensor_data2 = self.sensor_data2.sort_values(["reg_no","time_str"])
        self.sensor_data2['time_delta'] = self.sensor_data2.groupby(['reg_no'])['time_str'].diff()
        self.sensor_data2['time_delta'] = self.sensor_data2['time_delta'].apply(lambda x: x.total_seconds())
        self.sensor_data2.loc[self.sensor_data2.groupby(['reg_no','date'])['time_delta'].\
                        head(1).index, 'time_delta'] = np.NaN
        self.sensor_data2 = self.sensor_data2.sort_values(["reg_no","time_str"]).reset_index().drop(columns = {'index'})
        self.total_time_wk = self.sensor_data2.groupby(["reg_no","week"]).apply(lambda x: pd.Series({"drive_time": x["time_delta"].sum()})).\
                        reset_index()
        
        
        self.totalvio_rate_wk = self.totalvio_wk.merge(self.total_time_wk,on = ['reg_no','week'], how = 'outer')
        self.totalvio_rate_wk['count'] = self.totalvio_rate_wk['count'].fillna(0)
        #self.totalvio_drivetime_wk = self.totalvio_drivetime_wk[~(self.totalvio_drivetime_wk['drive_time'].isnull())].fillna(0)
        self.totalvio_rate_wk['drive_time_hr'] = self.totalvio_rate_wk['drive_time']/3600
        self.totalvio_rate_wk['vio_rate'] = self.totalvio_rate_wk['count']/self.totalvio_rate_wk['drive_time_hr']
        
        self.extvio_rate_wk = self.extvio_wk.merge(self.total_time_wk,on = ['reg_no','week'], how = 'outer')
        self.extvio_rate_wk['count'] = self.extvio_rate_wk['count'].fillna(0)
        #self.totalvio_drivetime_wk = self.totalvio_drivetime_wk[~(self.totalvio_drivetime_wk['drive_time'].isnull())].fillna(0)
        self.extvio_rate_wk['drive_time_hr'] = self.extvio_rate_wk['drive_time']/3600
        self.extvio_rate_wk['vio_rate'] = self.extvio_rate_wk['count']/self.extvio_rate_wk['drive_time_hr']
        
        


    
    
    
    def past_twowk_agg(self):
        self.start_date = date(self.year,self.month,self.date) - timedelta(days = 13)
        self.past2wk_eco_data = self.eco_data[(self.eco_data.begin_date <= date(self.year,self.month,self.date)) &\
                                             (self.eco_data.begin_date >= self.start_date)]
        self.past2wk_sensor_data = self.sensor_data[(self.sensor_data.date <= date(self.year,self.month,self.date)) &\
                                             (self.sensor_data.date >= self.start_date)]
        
        self.past2wk_sensor_data2 = self.sensor_data2[(self.sensor_data2.date <= date(self.year,self.month,self.date)) &\
                                             (self.sensor_data2.date >= self.start_date)]
        
        self.total_time_day = self.past2wk_sensor_data2.groupby(["reg_no","date"]).apply(lambda x: pd.Series({"drive_time": x["time_delta"].sum()})).\
                        reset_index()
        
        self.past2wk_totalvio = self.past2wk_eco_data
        #self.past2wk_totalvio['count'] = self.past2wk_totalvio.groupby(['reg_no','begin_date'])['violation'].transform(lambda x: x.count())
        #self.past2wk_totalvio = self.past2wk_totalvio.drop_duplicates(['reg_no','begin_date'])
        #self.past2wk_totalvio = self.past2wk_eco_data.groupby(['reg_no','begin_date']).size().\
        #to_frame('count').reset_index().rename(columns = {'begin_date': 'date'})
        self.past2wk_totalvio['label'] = ['total' for i in range(len(self.past2wk_totalvio))]
        
        self.past2wk_extvio = self.past2wk_eco_data[self.past2wk_eco_data.level == 'extreme']
        #self.past2wk_extvio = self.past2wk_eco_data[self.past2wk_eco_data.level == 'extreme'].\
        #groupby(['reg_no','begin_date']).size().\
        #to_frame('count').reset_index().rename(columns = {'begin_date': 'date'})
        #self.past2wk_extvio['count'] = self.past2wk_extvio.groupby(['reg_no','begin_date'])['violation'].transform(lambda x: x.count())
        #self.past2wk_extvio = self.past2wk_extvio.drop_duplicates(['reg_no','begin_date'])
        self.past2wk_extvio['label'] = ['extreme' for i in range(len(self.past2wk_extvio))]
        self.past2wk_totext_vio = self.past2wk_totalvio.append(self.past2wk_extvio).\
        sort_values(['reg_no','begin_date'])
        
        
        
        
        
        
        self.past2wk_sp_agg = self.past2wk_sensor_data.groupby(["reg_no","date","hr","day_hr"]).\
                        apply(lambda x: pd.Series({'sp_50th': x["speed_kmhr"].quantile(0.5),
                           'sp_85th': x['speed_kmhr'].quantile(0.85),
                           'sp_max': x['speed_kmhr'].max()})).reset_index()
        
        
        
        
        
    def boxplot_eco_past2wk(self, export = True):
        fig = go.Figure(go.Box(x = self.past2wk_eco_data.harsh_event.values,
                               y = self.past2wk_eco_data.value_g.values,
                               boxpoints='suspectedoutliers',
                               name = "Violation types"))
        fig.update_layout(title_text="Boxplot of acceleration values")
        
        if export:
            file_name = self.outputs_path + '/' + 'box_acc_past2wk.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    
    def boxplot_sensor_speed_past2wk(self, export = True):
        fig = go.Figure(go.Box(
                               y = self.past2wk_sensor_data.speed_kmhr.values,
                               boxpoints='suspectedoutliers',
                               name = "Speed"))
        fig.update_layout(title_text="Boxplot of speed values")
        
        if export:
            file_name = self.outputs_path + '/' + 'box_speed_past2wk.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    
    def totalext_vio_past2wk(self, export = True):
        fig = px.line(self.past2wk_totext_vio, x='begin_date', y='count', color = 'label',
                    facet_col ='reg_no', facet_col_wrap = 4, 
                    facet_row_spacing=0.04, 
                    facet_col_spacing=0.04, 
                    height=900, width=1000,
                    labels = {'begin_date':'Date','count':'Count'},
                    title= "Total number of violations for all vehicles")
        fig.update_yaxes(rangemode="tozero")
        #fig.update_yaxes(matches=None)
        #fig.add_trace(go.Scatter(), row='all', col='all', exclude_empty_subplots=True)
        
        if export:
            file_name = outputs_path + '/' + 'total_extvio_past2wk.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
        
        
    
    
    def hist_viotype_past2wk(self, export = True):
        self.viotype_cnt = self.past2wk_eco_data.groupby(['reg_no','harsh_event']).\
        size().to_frame('count').reset_index()
        
        fig = px.histogram(self.viotype_cnt, x="harsh_event", y = "count" ,color = "harsh_event", 
                   facet_col="reg_no",facet_col_wrap = 4,
                   labels = {"harsh_event":"Harsh event"},
                   title = "Number of violation types for all vehicles",
                   facet_row_spacing=0.04,facet_col_spacing=0.04,height=1000, width=1100)

        file_name = outputs_path + "/hist_viotypecnt_past2wk.html"
        print('Saving: ' + file_name)
        plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    
    def line_speed_past2wk(self, export = True):
        fig = px.line(self.past2wk_sp_agg, x="day_hr", y = "sp_50th",
                   facet_col="reg_no",facet_col_wrap = 4,
                   title = "50th percentile of speed each day each hour",
                   facet_row_spacing=0.04, facet_col_spacing=0.04, height=1000, width=1200
                  )

        file_name = outputs_path + '/line_50thspeed_hr_past2wk.html'
        print('Saving: ' + file_name)
        plotly.offline.plot(fig, filename = file_name, auto_open=False)
        
        
        fig2 = px.line(self.past2wk_sp_agg, x="day_hr", y = "sp_85th",
                   facet_col="reg_no",facet_col_wrap = 4,
                   title = "85th percentile of speed each day each hour",
                   facet_row_spacing=0.04, facet_col_spacing=0.04, height=1000, width=1200
                  )

        file_name = outputs_path + '/line_85thspeed_hr_past2wk.html'
        print('Saving: ' + file_name)
        plotly.offline.plot(fig2, filename = file_name, auto_open=False)
        
        
        fig3 = px.line(self.past2wk_sp_agg, x="day_hr", y = "sp_max",
                   facet_col="reg_no",facet_col_wrap = 4,
                   title = "Max speed each day each hour",
                   facet_row_spacing=0.04, facet_col_spacing=0.04, height=1000, width=1200
                  )

        file_name = outputs_path + '/line_maxspeed_hr_past2wk.html'
        print('Saving: ' + file_name)
        plotly.offline.plot(fig3, filename = file_name, auto_open=False)
        
        
        
    def print_eco_res(self):
        return self.eco_data
    
    def print_sensor_res(self):
        return self.sensor_data
    
    def print_sensor2wk_res(self):
        return self.past2wk_sensor_data
    
    def print_eco2wk_res(self):
        return self.past2wk_eco_data
    
    def print_spagg(self):
        return self.past2wk_sp_agg
    
    def print_past2wk_totalvio(self):
        return self.past2wk_totalvio, self.past2wk_extvio, self.total_time_day
    
    def boxplot_eco(self, export = True):
        fig = go.Figure(go.Box(x = self.eco_data.harsh_event.values,
                               y = self.eco_data.value_g.values,
                               boxpoints='suspectedoutliers',
                               name = "Violation types"))
        fig.update_layout(title_text="Boxplot of acceleration values")
        
        if export:
            file_name = self.outputs_path + '/' + 'box_acc_alldata.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    def boxplot_sensor_speed_wkwed(self, export = True):
        self.sensor_data['weekday'] = np.where(self.sensor_data['time_str'].dt.dayofweek < 5,"weekday","weekend")
        #spd_hr_wkwed = self.sensor_data.groupby(['hr','weekday']).size().to_frame('count').reset_index()
        fig = px.box(self.sensor_data, x="hr", y="speed_kmhr",facet_col="weekday", facet_col_wrap=2,
             width=1000, height=800)
        
        fig.update_layout(title_text="Boxplot of speed at each hour separated by weekday and weekend")
        
        if export:
            file_name = self.outputs_path + '/' + 'box_speed_wkwed_alldata.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    def hist_totalvio_wk(self, export = True):
        #totalvio_wk = self.eco_data.groupby(["reg_no","week"]).size().to_frame('count').reset_index()
        fig = px.histogram(self.totalvio_wk, x="week", y = "count" ,color = "week", 
                   facet_col="reg_no",facet_col_wrap = 4,
                    #labels = {"avg_cnt":"avg cnt","harsh_event":"Harsh event"},
                   facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
                   facet_col_spacing=0.04, # default is 0.03
                   height=1000, width=1100)
        
        fig.update_layout(title_text="Number of violation per week for all vehicles")
        
        if export:
            file_name = self.outputs_path + '/' + 'hist_totvio_wk_alldata.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    def hist_extvio_wk(self, export = True):
        #extvio_df = self.eco_data[self.eco_data.level == "extreme"]
        #extvio_wk = extvio_df.groupby(["reg_no","week"]).size().to_frame('count').reset_index()
        fig = px.histogram(self.extvio_wk, x="week", y = "count" ,color = "week", 
                   facet_col="reg_no",facet_col_wrap = 4,
                    #labels = {"avg_cnt":"avg cnt","harsh_event":"Harsh event"},
                   facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
                   facet_col_spacing=0.04, # default is 0.03
                   height=1000, width=1100)
        
        fig.update_layout(title_text="Number of extreme violation per week for all vehicles")
        
        if export:
            file_name = self.outputs_path + '/' + 'hist_extvio_wk_alldata.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    def hist_totalvio_rate_wk(self, export = True):
        fig = px.histogram(self.totalvio_rate_wk, x="week",y ="vio_rate", facet_col="reg_no",
                           facet_col_wrap = 5, color = 'week',
             labels = {"vio_rate": "Violation rate"},
             title = "Violation rate per week for all Vehicles",
             facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              facet_col_spacing=0.04, # default is 0.03
              height=900, width=1000)
        #fig.update_yaxes(matches=None)

        file_name = outputs_path + "/hist_totviorate_wk_allvehicles.html"
        print('Saving: ' + file_name)
        plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    def hist_extvio_rate_wk(self, export = True):
        fig = px.histogram(self.extvio_rate_wk, x="week",y ="vio_rate", facet_col="reg_no",
                           facet_col_wrap = 5, color = 'week',
             labels = {"vio_rate": "Violation rate"},
             title = "Extreme violation rate per week for all Vehicles",
             facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              facet_col_spacing=0.04, # default is 0.03
              height=900, width=1000)
        #fig.update_yaxes(matches=None)

        file_name = outputs_path + "/hist_extviorate_wk_allvehicles.html"
        print('Saving: ' + file_name)
        plotly.offline.plot(fig, filename = file_name, auto_open=False)
    
    
